# Continue Pretraining on T5

In [1]:
from datasets import load_dataset, concatenate_datasets

mont = load_dataset('billingsmoore/montano-bo-es', split='train')

lh = load_dataset('billingsmoore/tibetan-to-spanish-translation-dataset', split='train').rename_columns({'tibetan':'bo', 'spanish':'es'}).remove_columns(['phonetic'])

dataset = concatenate_datasets([mont, lh])

## Corrupt Training Text

T5 is trained by learning to correct missing spans in text. Thus, the training data must have spans masked for training.

In [2]:
import random

def bo_corrupt_text(example):
    text = example["bo"]
    words = text.split()
    num_masks = max(1, len(words) // 6)  # Mask ~15-20% of the words
    masked_indices = sorted(random.sample(range(len(words)), num_masks))

    new_text = []
    labels = []
    current_mask = 0

    for i, word in enumerate(words):
        if i in masked_indices:
            if not new_text or new_text[-1] != f"<extra_id_{current_mask}>":
                new_text.append(f"<extra_id_{current_mask}>")
                labels.append(f"<extra_id_{current_mask}> {word}")
                current_mask += 1
            else:
                labels[-1] += f" {word}"
        else:
            new_text.append(word)

    return {"input_text": " ".join(new_text), "target_text": " ".join(labels)}

bo_train_dataset = dataset.map(bo_corrupt_text)


Map:   0%|          | 0/21894 [00:00<?, ? examples/s]

In [3]:
def es_corrupt_text(example):
    text = example["es"]
    words = text.split()
    num_masks = max(1, len(words) // 6)  # Mask ~15-20% of the words
    masked_indices = sorted(random.sample(range(len(words)), num_masks))

    new_text = []
    labels = []
    current_mask = 0

    for i, word in enumerate(words):
        if i in masked_indices:
            if not new_text or new_text[-1] != f"<extra_id_{current_mask}>":
                new_text.append(f"<extra_id_{current_mask}>")
                labels.append(f"<extra_id_{current_mask}> {word}")
                current_mask += 1
            else:
                labels[-1] += f" {word}"
        else:
            new_text.append(word)

    return {"input_text": " ".join(new_text), "target_text": " ".join(labels)}

en_train_dataset = dataset.map(es_corrupt_text)

Map:   0%|          | 0/21894 [00:00<?, ? examples/s]

In [4]:
from datasets import concatenate_datasets

ds = concatenate_datasets([en_train_dataset, bo_train_dataset])

In [5]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('bo-es-tokenizer')

## Tokenize the Data for Training

In [7]:
def tokenize_data(example):
    inputs = tokenizer(example["input_text"], max_length=512, truncation=True, padding="max_length")
    targets = tokenizer(example["target_text"], max_length=512, truncation=True, padding="max_length")
    return {
        "input_ids": inputs.input_ids,
        "attention_mask": inputs.attention_mask,
        "labels": targets.input_ids
    }

tokenized_dataset = ds.map(tokenize_data, batched=True, remove_columns=["bo", 'es', "input_text", "target_text"])

Map:   0%|          | 0/43788 [00:00<?, ? examples/s]

## Train the Model

In [8]:
from transformers import TrainingArguments, Trainer, T5ForConditionalGeneration

model = T5ForConditionalGeneration.from_pretrained("google-t5/t5-base", device_map='cuda:0')
model.resize_token_embeddings(len(tokenizer))

training_args = TrainingArguments(
    output_dir="es-continue-pretrain",
    save_strategy="epoch",
    auto_find_batch_size=True,
    learning_rate=3e-4,
    num_train_epochs=3,
    push_to_hub=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: billingsmoore. Use `wandb login --relogin` to force relogin


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss
500,0.109800
1000,0.052500
1500,0.051200
2000,0.049400
2500,0.047200
3000,0.053600
3500,0.049600
4000,0.047700
4500,0.048400
5000,0.048500


KeyboardInterrupt: 

In [9]:
model.save_pretrained('es-pretrained-model')